<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [3]:
#ANSWER:
print(response.status_code)

200


In [4]:
#ANSWER
print(response.text)

{"message": "success", "iss_position": {"latitude": "-30.1933", "longitude": "47.5401"}, "timestamp": 1706167014}


In [11]:
payload = response.json()
payload['iss_position']['latitude']

'-30.1933'

We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [ ]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [12]:
#ANSWER:
print(response.headers)

{'Server': 'nginx/1.10.3', 'Date': 'Thu, 25 Jan 2024 07:16:54 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'access-control-allow-origin': '*'}


Print the content of the response (the data that the server returned):

In [19]:
#ANSWER:
print(response.content)

b'{"message": "success", "iss_position": {"latitude": "-30.1933", "longitude": "47.5401"}, "timestamp": 1706167014}'


Note that this is a Python byte string:

In [17]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [15]:
#ANSWER:
print(response.headers["content-type"])

application/json


JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [22]:
overheads = response.json()
print(overheads)

{'message': 'success', 'iss_position': {'latitude': '-30.1933', 'longitude': '47.5401'}, 'timestamp': 1706167014}


What kind of object did this give us?

In [21]:
#ANSWER:
print(type(overheads))

<class 'dict'>


In [23]:
overheads.keys()

dict_keys(['message', 'iss_position', 'timestamp'])

In [24]:
overheads.values()

dict_values(['success', {'latitude': '-30.1933', 'longitude': '47.5401'}, 1706167014])

In [26]:
import pandas as pd
pd.DataFrame(overheads)

,message,iss_position,timestamp
latitude,success,-30.1933,1706167014
longitude,success,47.5401,1706167014


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate list:

In [20]:
#ANSWER:
passes = overheads['passes']

KeyError: 'passes'

Now extract the `start` strings into an array called `srisetimes`:

In [ ]:
#ANSWER:
srisetimes = [xpass['start'] for xpass in passes]
srisetimes

['2022-10-18T07:05:28.173Z',
 '2022-10-18T08:41:53.173Z',
 '2022-10-18T10:20:58.173Z',
 '2022-10-18T12:00:28.173Z',
 '2022-10-18T13:37:18.173Z',
 '2022-10-18T15:13:28.173Z',
 '2022-10-18T16:50:53.173Z',
 '2022-10-19T06:17:43.173Z']

These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [ ]:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]

In [ ]:
risetimes

[datetime.datetime(2022, 10, 18, 7, 5, 28, 173000),
 datetime.datetime(2022, 10, 18, 8, 41, 53, 173000),
 datetime.datetime(2022, 10, 18, 10, 20, 58, 173000),
 datetime.datetime(2022, 10, 18, 12, 0, 28, 173000),
 datetime.datetime(2022, 10, 18, 13, 37, 18, 173000),
 datetime.datetime(2022, 10, 18, 15, 13, 28, 173000),
 datetime.datetime(2022, 10, 18, 16, 50, 53, 173000),
 datetime.datetime(2022, 10, 19, 6, 17, 43, 173000)]

Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```

In [ ]:
#ANSWER:
for risetime in risetimes:
    print(risetime.strftime('%d/%m/%y %I:%M'))

18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
18/10/22 04:50
19/10/22 06:17


Here is an endpoint that tells us who is onboard:

In [27]:
response = requests.get("http://api.open-notify.org/astros.json")

In [28]:
a = response.json()
a

{'message': 'success',
 'people': [{'name': 'Jasmin Moghbeli', 'craft': 'ISS'},
  {'name': 'Andreas Mogensen', 'craft': 'ISS'},
  {'name': 'Satoshi Furukawa', 'craft': 'ISS'},
  {'name': 'Konstantin Borisov', 'craft': 'ISS'},
  {'name': 'Oleg Kononenko', 'craft': 'ISS'},
  {'name': 'Nikolai Chub', 'craft': 'ISS'},
  {'name': "Loral O'Hara", 'craft': 'ISS'}],
 'number': 7}

Referring to the methods used above, extract the number of astronauts and their names:

In [ ]:
#ANSWER:
astros = response.json()
print(astros)
print(astros["number"])
for astronaut in astros['people']:
    print(astronaut['name']) 

{'message': 'success', 'people': [{'name': 'Cai Xuzhe', 'craft': 'Tiangong'}, {'name': 'Chen Dong', 'craft': 'Tiangong'}, {'name': 'Liu Yang', 'craft': 'Tiangong'}, {'name': 'Sergey Prokopyev', 'craft': 'ISS'}, {'name': 'Dmitry Petelin', 'craft': 'ISS'}, {'name': 'Frank Rubio', 'craft': 'ISS'}, {'name': 'Nicole Mann', 'craft': 'ISS'}, {'name': 'Josh Cassada', 'craft': 'ISS'}, {'name': 'Koichi Wakata', 'craft': 'ISS'}, {'name': 'Anna Kikina', 'craft': 'ISS'}], 'number': 10}
10
Cai Xuzhe
Chen Dong
Liu Yang
Sergey Prokopyev
Dmitry Petelin
Frank Rubio
Nicole Mann
Josh Cassada
Koichi Wakata
Anna Kikina


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [ ]:
#ANSWER:
def handleResponse(response, verbose = False):
    if response.status_code == 200:
        if verbose:
            print('Response = 200: OK')
        fail = False
    else:
        if verbose:
            print('Response = ' + str(response.status_code) + ': ' + '(TODO: output of lookup function here)')
        fail = True
    return(fail, response.status_code)

2. Test your response handler on some correct and incorrect API calls.

In [ ]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)[0]

Response = 200: OK


False



---



---



> > > > > > > > > © 2023 Institute of Data


---



---



